In [8]:
#импортируем библиотеки
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup

import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime

from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import pandas as pd
import openpyxl

# Работа с моделью

## Подготовка данных для обучения

### Загрузка данных для Россия/не Россия

In [37]:
#загрузка данных для обучения Россия/не Россия
df = pd.read_excel('real_inv.xlsx',sheet_name = 'country')
df.country.value_counts()

country
1    2572
0    1116
Name: count, dtype: int64

In [38]:
df

,clean_message,country
0,"Разбор Alibaba Alibaba (тикер ВАВА, доступна ...",0
1,Итоги дня 25 июля ? Выручка Positive Techn...,1
2,"Вот и негатив про Tether (USDT) подъехал, не п...",0
3,Франция и Германия приостановили использование...,0
4,Крупнейший в России собственник самолетов Гос...,1
...,...,...
3683,Российские кофейни начали использовать ИИ для ...,1
3684,Посольство России в США после публикации о Сев...,0
3685,В России появится ипотека под 0. Беспроцентные...,1
3686,Путин и глава Евросовета обсудили возможности ...,1


In [27]:
# Разделение на train и validation
train_data, valid_data = train_test_split(df, test_size=0.2, random_state=42, stratify = df['country'])

X_train=list(train_data['clean_message'])
y_train=list(train_data['countrдляy'])
X_valid=list(valid_data['clean_message'])
y_valid=list(valid_data['real_inv'])

### Загрузка данных Реальные инвестиции или нет

In [43]:
#загрузка данных для обучения Россия/не Россия
df = pd.read_excel('real_inv.xlsx',sheet_name = 'real')
df.real.value_counts()

real
0    3077
1     678
Name: count, dtype: int64

In [44]:
df

,clean_message,real
0,"Разбор Alibaba Alibaba (тикер ВАВА, доступна ...",0
1,Итоги дня 25 июля ? Выручка Positive Techn...,0
2,"Вот и негатив про Tether (USDT) подъехал, не п...",0
3,Франция и Германия приостановили использование...,0
4,Крупнейший в России собственник самолетов Гос...,0
...,...,...
3750,Российские кофейни начали использовать ИИ для ...,0
3751,Посольство России в США после публикации о Сев...,1
3752,В России появится ипотека под 0. Беспроцентные...,1
3753,Путин и глава Евросовета обсудили возможности ...,1


In [45]:
# Разделение на train и validation
train_data, valid_data = train_test_split(df, test_size=0.2, random_state=42, stratify = df['real'])

X_train=list(train_data['clean_message'])
y_train=list(train_data['real'])
X_valid=list(valid_data['clean_message'])
y_valid=list(valid_data['real'])

## Обучение модели

In [47]:
# Загрузка предварительно обученной модели BERT для задачи классификации последовательностей
model_type = 'cointegrated/rubert-tiny2'
model = BertForSequenceClassification.from_pretrained(model_type)

# Загрузка токенизатора, связанного с выбранной моделью BERT
tokenizer = BertTokenizer.from_pretrained(model_type)

# Определение устройства (GPU, если доступен, иначе CPU) для обучения модели
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Путь для сохранения обученной модели, в названии основные параметры для понимания при большом количестве экспериментов
model_save_path = 'content/bert2+f1_1024_REAL_RUS_weight_0.15,1_rand42_01.04.pt' 

# Максимальная длина входной последовательности, обрезается или дополняется токенами до этой длины
max_len = 1024

# Количество эпох обучения
epochs = 15

n_classes=2

# Извлечение размерности выхода из второго слоя BERT
out_features = model.bert.encoder.layer[1].output.dense.out_features

# Замена классификационного слоя на линейный слой с двумя выходами (предполагается бинарная классификация)
model.classifier = torch.nn.Linear(out_features, n_classes)
# Перемещение модели на выбранное устройство (GPU или CPU)
model.to(device)
print("ok")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ok


In [135]:
# определение необходимых классов (определено документацией)
class CustomDataset(Dataset):

  def __init__(self, texts, targets, tokenizer, max_len=512):
    self.texts = texts
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, idx):
    text = str(self.texts[idx])
    target = self.targets[idx]

    encoding = self.tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=self.max_len,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
        truncation=True
    )

    return {
      'text': text,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }

In [136]:
# Создание обучающего датасета с использованием кастомного класса CustomDataset
train_set = CustomDataset(X_train, y_train, tokenizer)

# Создание валидационного датасета с использованием кастомного класса CustomDataset
valid_set = CustomDataset( X_valid, y_valid, tokenizer)

# Создание обучающего DataLoader для обеспечения подачи данных в модель пакетами
train_loader = DataLoader(train_set, batch_size=2, shuffle=True)

# Создание валидационного DataLoader для обеспечения подачи данных в модель пакетами
valid_loader = DataLoader(valid_set, batch_size=2, shuffle=True)

# Инициализация оптимизатора AdamW для обновления параметров модели в процессе обучения
# при lr=2e-5 быстро переобучается, надо уменьшать
optimizer = AdamW(model.parameters(), lr=7e-6, correct_bias=False, no_deprecation_warning=True)

# Инициализация планировщика обучения для управления тем, как изменяется скорость обучения в течение эпохи
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=len(train_loader) * 2,  # эпохи от общего числа шагов обучения, подбирать
    num_training_steps=len(train_loader) * epochs
)
# Задайте веса для каждого класса, например, повысим вес первого класса. Для россия/не россия [1, 0.5]
class_weights = torch.tensor([1, 0.5]) # Россия
#class_weights = torch.tensor([0.15, 1.0]) # инвестиции


# Инициализация функции потерь для задачи классификации (в данном случае, CrossEntropyLoss)
loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights).to(device)

In [137]:
#Обучение модели
from sklearn.metrics import confusion_matrix

best_f1 = 0

train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

# Цикл по эпохам
for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}', end = " ")
    print(datetime.now().time())
    
    # Установка модели в режим обучения
    model = model.train()
    losses = []
    correct_predictions = 0
    i=0
    len_train = len(train_loader)//20
    # Цикл по данным обучения
    for data in train_loader:
        i+=1
        if i%len_train == 0:
            print("|", end = "")
        # Извлечение данных из DataLoader
        input_ids = data["input_ids"].to(device)
        attention_mask = data["attention_mask"].to(device)
        targets = data["targets"].to(device)

        # Получение предсказаний модели и вычисление функции потерь
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        preds = torch.argmax(outputs.logits, dim=1)
        loss = loss_fn(outputs.logits, targets)

        # Вычисление точности предсказаний
        correct_predictions += torch.sum(preds == targets)

        # Сохранение потерь для текущей итерации
        losses.append(loss.item())
        
        # Обратное распространение ошибки и оптимизация параметров
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    # Вычисление и вывод средних потерь и точности на обучающем наборе
    train_acc = correct_predictions.double() / len(train_set)
    train_loss = np.mean(losses)
    #*****  
    
    print(" ")
    print(f'Train loss {train_loss} accuracy {train_acc}')

    # Установка модели в режим оценки
    model = model.eval()
    losses = []
    correct_predictions = 0

    from sklearn.metrics import f1_score

    # Оценка модели на валидационном наборе
    with torch.no_grad():
        true_labels = []
        predicted_labels = []
        for data in valid_loader:
            input_ids = data["input_ids"].to(device)
            attention_mask = data["attention_mask"].to(device)
            targets = data["targets"].to(device)
    
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
    
            preds = torch.argmax(outputs.logits, dim=1)
    
            true_labels.extend(targets.cpu().numpy())
            predicted_labels.extend(preds.cpu().numpy())
    
            loss = loss_fn(outputs.logits, targets)
            correct_predictions += torch.sum(preds == targets)
            losses.append(loss.item())
    

            
    # Вычисление и вывод средних потерь и точности на валидационном наборе
    val_acc = correct_predictions.double() / len(valid_set)
    val_loss = np.mean(losses)
    # Вычисление F1-score
    #f1score = f1_score(true_labels, predicted_labels, average='macro')
    f1score = f1_score(true_labels, predicted_labels, average='binary')
    val_accuracy = accuracy_score(true_labels, predicted_labels)
    print(f'Val loss {val_loss} accuracy {val_acc} F1-score: {f1score} accuracy2 {val_accuracy}')real_inv
    conf_matrix = confusion_matrix(true_labels, predicted_labels)
    print(conf_matrix)
    
    print('-' * 10)

    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    # Сохранение модели, если текущая точность на валидации превышает лучшую точность
    if f1score > best_f1:
        torch.save(model, model_save_path[:-3] + ' epoch-' + str(epoch) + '_f1_'+ str(f1score)[:5] + '.ph' )
        best_f1 = f1score
        best_epoch = epoch
# сохраняются все модели имеющие лучший результат по сравнению с предыдущим, поэтому занимают много места. 
# При отсутствии такой необходимости отредактировать код

Epoch 1/15 15:38:42.082805
|||||||||||||||||||| 
Train loss 0.854396310259733 accuracy 0.8646333104866347
Val loss 0.8749401989067611 accuracy 0.8684931506849315 F1-score: 0.46480938416422285 accuracy2 0.8684931506849315
[[634   0]
 [ 96   0]]
----------
Epoch 2/15 15:39:22.362958
|||||||||||||||||||| 
Train loss 0.7241596640962176 accuracy 0.8749143248800549
Val loss 0.6417888398896201 accuracy 0.8904109589041096 F1-score: 0.65746996996997 accuracy2 0.8904109589041096
[[626   8]
 [ 72  24]]
----------
Epoch 3/15 15:40:02.212522
|||||||||||||||||||| 
Train loss 0.5298525400307286 accuracy 0.9115832762165867
Val loss 0.5880769997745057 accuracy 0.9013698630136986 F1-score: 0.789655669211929 accuracy2 0.9013698630136986
[[595  39]
 [ 33  63]]
----------
Epoch 4/15 15:40:41.574493
|||||||||||||||||||| 
Train loss 0.48189277370707123 accuracy 0.9283755997258397
Val loss 0.608393254806168 accuracy 0.8917808219178083 F1-score: 0.7758429695850744 accuracy2 0.8917808219178082
[[588  46]
 [ 33 